In [1]:
import tensorflow.keras as keras
import segmentation_models as sm
sm.set_framework('tf.keras')
import random
import re
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil
from pathlib import Path
import tensorflow as tf
from tensorflow.keras import datasets, utils, callbacks, optimizers, losses
from keras.preprocessing.image import ImageDataGenerator

import ridurre
from example.cifar_10_resnet import resnet
from ridurre import model_complexity

N_CLASSES = 1
LR = 0.0001


print('[LOAD DATA FROM DISK]')


def split_set(set_, k_train, k_val, k_test):
    quantity = len(set_)
    random.shuffle(set_)
    train_quantity = round(quantity*k_train)
    val_quatity= round(quantity*k_val)

    train = set_[0:train_quantity]
    val = set_[train_quantity:train_quantity+val_quatity]
    test= set_[-1:train_quantity+val_quatity-1:-1]

    return train, val, test


def sync_test_redundant_images(path_to_images, images, annotations):
    images_sync = []
    for annot_image in annotations:
        annot_image = re.split('[./\\\]', annot_image)[-2]
        images_sync.append(os.path.join(path_to_images,'train',annot_image + '.tif'))
    return images_sync, annotations




def load_train_annot_dataset(path_to_images, k_train=0.85, k_val=0.1, k_test=0.05):
    train_paths = glob.glob(os.path.join(path_to_images, 'train', '*'))
    annot_paths = glob.glob(os.path.join(path_to_images, 'annot', '*'))

#     train_paths =  os.listdir(os.path.join(path_to_images, 'train'))
#     annot_paths = os.listdir(os.path.join(path_to_images, 'annot'))

    train_annotations, val_annotations, test_annotations = split_set(annot_paths, k_train, k_val, k_test)
    train_images, val_images, test_images = split_set(train_paths, k_train, k_val, k_test)

    train_images, train_annotations = sync_test_redundant_images(path_to_images, train_images, train_annotations)
    test_images, test_annotations = sync_test_redundant_images(path_to_images,test_images, test_annotations)
    val_images, val_annotations = sync_test_redundant_images(path_to_images,val_images, val_annotations)

    assert len(annot_paths) == len(train_annotations) + len(val_annotations) + len(test_annotations)




    return train_images, train_annotations, val_images, val_annotations, test_images, test_annotations


Using TensorFlow backend.



Segmentation Models: using `keras` framework.

[LOAD DATA FROM DISK]


In [2]:
DATA_DIR = 'D:\Study\Skolkovo_Immersion\matching'


In [3]:
train_images, train_annotations, val_images, val_annotations, \
test_images, test_annotations = load_train_annot_dataset(DATA_DIR)

print('[DATA LOADED FROM DISK]')

# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(30, 30))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()

# helper function for data visualization
def denormalize(x):
    """Scale image to range 0..1 for correct plot"""
    x_max = np.percentile(x, 98)
    x_min = np.percentile(x, 2)
    x = (x - x_min) / (x_max - x_min)
    x = x.clip(0, 1)
    return x


# classes for data loading and preprocessing
class Dataset:
    """CamVid Dataset. Read images, apply augmentation and preprocessing transformations.

    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing
            (e.g. noralization, shape manipulation, etc.)

    """

    CLASSES = ['leaf']

    def __init__(
            self,
            images,
            masks,
            classes=None,
            augmentation=None,
            preprocessing=None,
    ):

        self.masks_fps = masks
        self.images_fps = images
        self.ids = len(self.masks_fps) #if len(self.masks_fps) == len(self.images_fps)
        # convert str names to class values on masks4
        self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]

        self.augmentation = augmentation
        self.preprocessing = preprocessing

    def __getitem__(self, i):

        # read data
        image = cv2.imread(self.images_fps[i], -1)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.masks_fps[i], 0)
        # extract certain classes from mask (e.g. cars)
        masks = [(mask == v) for v in self.class_values]
        mask = np.stack(masks, axis=-1).astype('float')

        # add background if mask is not binary
        if mask.shape[-1] != 1:
            background = 1 - mask.sum(axis=-1, keepdims=True)
            mask = np.concatenate((mask, background), axis=-1)

        # apply augmentations
        if self.augmentation:

            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']

        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']

        return image, mask

    def __len__(self):
        return self.ids


class Dataloder(keras.utils.Sequence):
    """Load data from dataset and form batches

    Args:
        dataset: instance of Dataset class for image loading and preprocessing.
        batch_size: Integet number of images in batch.
        shuffle: Boolean, if `True` shuffle image indexes each epoch.
    """

    def __init__(self, dataset, batch_size=1, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(dataset))

        self.on_epoch_end()

    def __getitem__(self, i):

        # collect batch data
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        X, Y = [], []
        for j in range(start, stop):
            data = self.dataset[j]
            X.append(data[0])
            Y.append(data[1])
        # transpose list of lists
        X = np.array(X)
        Y = np.array(Y)
        ######
        if len(X.shape) < 4:
            X = np.expand_dims(X, axis=3)
        return X, Y

    def __len__(self):
        """Denotes the number of batches per epoch"""
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        """Callback function to shuffle indexes each epoch"""
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)

import albumentations as A

def round_clip_0_1(x, **kwargs):
    return x.round().clip(0, 1)

# define heavy augmentations
def get_training_augmentation():
    train_transform = [

        A.HorizontalFlip(p=0.5),
#         A.VerticalFlip(p=0.1),
        A.ShiftScaleRotate(scale_limit=(-0.2,0.2), rotate_limit=(-15,15), shift_limit=(-0.01,0.01), p=0.7, border_mode=cv2.BORDER_CONSTANT, mask_value=1),
        A.RandomCrop(height=640, width=960, always_apply=True),
        # A.RandomCrop(height=400, width=560, p=0.3),
        A.PadIfNeeded(min_height=640, min_width=960, always_apply=True, border_mode=cv2.BORDER_CONSTANT, mask_value=1),

#         A.IAAAdditiveGaussianNoise(p=0.2),
        A.IAAPerspective(p=0.5),

#         A.OneOf(
#             [
#                 A.CLAHE(p=1),
#                 A.RandomBrightness(p=0.2),
#                 A.RandomGamma(p=0.2),
#             ],
#             p=0.3,
#         ),

#         A.OneOf(
#             [
#                 A.IAASharpen(p=0.5),
#                 A.Blur(blur_limit=3, p=0.5),
#                 A.MotionBlur(blur_limit=3, p=0.5),
#             ],
#             p=0.3,
#         ),


        A.RandomContrast(limit=(-0.1, 0.1), p=0.2),
#                 A.HueSaturationValue(p=1),

        A.Lambda(mask=round_clip_0_1)
    ]
    return A.Compose(train_transform)


def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        A.RandomCrop(height=640, width=960, always_apply=True)
    ]
    return A.Compose(test_transform)

def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform

    Args:
        preprocessing_fn (callbale): data normalization function
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose

    """

    _transform = [
        A.Lambda(image=preprocessing_fn),
    ]
    return A.Compose(_transform)

print('[CREATING DATASET]')

BACKBONE = 'efficientnetb3'
BATCH_SIZE = 1
CLASSES = ['leaf']
LR = 0.0001
EPOCHS = 30


train_dataset = Dataset(
    train_images,
    train_annotations,
    classes=CLASSES,
    augmentation=get_training_augmentation(),
#     preprocessing=get_preprocessing(preprocess_input),
)

# Dataset for validation images
valid_dataset = Dataset(
    val_images,
    val_annotations,
    classes=CLASSES,
    augmentation=get_validation_augmentation(),
#     preprocessing=get_preprocessing(preprocess_input),
)

train_dataloader = Dataloder(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = Dataloder(valid_dataset, batch_size=1, shuffle=False)

print('[DATASET CREATED]')

[DATA LOADED FROM DISK]
[CREATING DATASET]
[DATASET CREATED]


In [4]:
print('[LOAD MODEL FROM DISK]')


def load_model():
    # with open("path/to/json", 'r') as json_file:
    #     loaded_model_json = json_file.read()
    # loaded_model = model_from_json(loaded_model_json)
    # # load weights into new model
    # loaded_model.load_weights("path/to/h5")

    loaded_model = keras.models.load_model('pruned_model.h5', compile=False)

    return loaded_model

model = load_model()

#saving pretrained conv layer for 1->3 channels transform
pretrained_first_layer_weights = model.layers[1].get_weights()


model.summary()

print('[MODEL LOADED FROM DISK]')

[LOAD MODEL FROM DISK]
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 640, 960, 1)]     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 640, 960, 3)       6      

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [5]:
def compile_model(model):

    inp_layer = keras.Input(shape=(None, None, 1))
    l1 = keras.layers.Conv2D(3, (1, 1))(inp_layer)

    out = model(l1)

    new_model = keras.Model(inp_layer, out)

    new_model.layers[1].set_weights(pretrained_first_layer_weights)

    optim = keras.optimizers.Adam(LR)

    # Segmentation models losses can be combined together by '+' and scaled by integer or float factor
    dice_loss = sm.losses.DiceLoss()
    focal_loss = sm.losses.BinaryFocalLoss() if N_CLASSES == 1 else sm.losses.CategoricalFocalLoss()
    total_loss = dice_loss + (1 * focal_loss)

    metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]
    new_model.compile(optim, total_loss, metrics)
    return new_model

In [6]:
print('[PREPARING FOR TRAINING]')

TRAIN_LOGS_FOLDER_PATH = Path("./train_logs")

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(str(TRAIN_LOGS_FOLDER_PATH) + "/model_{epoch:02d}.h5",
                                                      save_best_only=False,
                                                      save_weights_only=False,
                                                      verbose=1
                                                      )

callbacks = [model_checkpoint_callback]

# Train the model
FIRST_TRAIN_EPOCHS = 1
BATCH_SIZE = 32
STEPS_PER_EPOCH = len(train_dataloader)
STEPS_VALID = len(valid_dataloader)


[PREPARING FOR TRAINING]


In [7]:
print('[COMPILING THE MODEL]')

optim = keras.optimizers.Adam(LR)

# Segmentation models losses can be combined together by '+' and scaled by integer or float factor
dice_loss = sm.losses.DiceLoss()
focal_loss = sm.losses.BinaryFocalLoss() if N_CLASSES == 1 else sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]
model.compile(optim, total_loss, metrics)

[COMPILING THE MODEL]


In [8]:
print('[TRAINING THE MODEL]')

model.fit_generator(train_dataloader,
                    steps_per_epoch=STEPS_PER_EPOCH,
                    epochs=FIRST_TRAIN_EPOCHS,
                    validation_data=valid_dataloader,
                    callbacks=callbacks,
                    validation_steps=STEPS_VALID
                    )

print('[MODEL PRETRAINED]')


[TRAINING THE MODEL]
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
 31/260 [==>...........................] - ETA: 58s - loss: 0.0521 - iou_score: 0.9681 - f1-score: 0.9835    

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
print('[SEGMENTATION]')

test_dataset = Dataset(
    test_images,
    test_annotations,
    classes=CLASSES,
    augmentation=get_validation_augmentation(),
    #preprocessing=get_preprocessing(preprocess_input),
)

test_dataloader = Dataloder(test_dataset, batch_size=1, shuffle=False)

ids = np.arange(len(test_dataset))


for i in ids:
    image, gt_mask = test_dataset[i]
    image = np.expand_dims(image, axis=0)
    image = np.expand_dims(image, axis=3)
    pr_mask = model.predict(image).round()
# img_kek = train_dataloader[0][0]
# pr_mask = model.predict(img_kek).round()
# plt.figure(figsize = (28,28))
# plt.imshow(img_kek[..., 0].squeeze(),cmap='gray')
# plt.show()
# print(np.max(img_kek))
#     cv2.imshow('mask', image[0][:][:])
#     cv2.waitKey(0)

    visualize(
        image=denormalize(image.squeeze()),
        gt_mask=gt_mask[..., 0].squeeze(),
        pr_mask=pr_mask[..., 0].squeeze(),
    )

In [9]:
print('[CRATING MODEL FOR PRUNING]')

BACKBONE = 'efficientnetb3'

n_classes = 1 if len(CLASSES) == 1 else (len(CLASSES) + 1)  # case for binary and multiclass segmentation
activation = 'sigmoid' if n_classes == 1 else 'softmax'

base_model = sm.Unet(BACKBONE, classes=n_classes, activation=activation, input_shape=(None, None, 3))

base_model.set_weights(model.layers[-1].get_weights())

model_for_pruning = base_model

[CRATING MODEL FOR PRUNING]


In [15]:
print('[START PRUNING]')

import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 2
epochs = 2

end_step = np.ceil(len(train_images) / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

sub_model_for_prun = prune_low_magnitude(model_for_pruning, **pruning_params)

# `prune_low_magnitude` requires a recompile.


sub_model_for_prun.summary()


[START PRUNING]
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
prune_low_magnitude_stem_conv ( (None, None, None, 4 2162        input_1[0][0]                    
__________________________________________________________________________________________________
prune_low_magnitude_stem_bn (Pr (None, None, None, 4 161         prune_low_magnitude_stem_conv[0][
__________________________________________________________________________________________________
prune_low_magnitude_stem_activa (None, None, None, 4 1           prune_low_magnitude_stem_bn[0][0]
______________________________________________________________________________

In [16]:
model_for_prun = compile_model(sub_model_for_prun)
model_for_prun.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None, None, 1)]   0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 3)     6         
_________________________________________________________________
model (Model)                (None, None, None, 1)     35166037  
Total params: 35,166,043
Trainable params: 17,778,559
Non-trainable params: 17,387,484
_________________________________________________________________


In [17]:
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    # Log sparsity and other metrics in Tensorboard.
    tfmot.sparsity.keras.PruningSummaries(log_dir=str(TRAIN_LOGS_FOLDER_PATH)),
    tf.keras.callbacks.ModelCheckpoint(str(TRAIN_LOGS_FOLDER_PATH) + "/pruning_model_{epoch:02d}.h5",
                                                      save_best_only=False,
                                                      save_weights_only=False,
                                                      verbose=1
                                                      )
]

model_for_prun.fit_generator(train_dataloader,
                    steps_per_epoch=STEPS_PER_EPOCH,
                    epochs=2,
                    validation_data=valid_dataloader,
                    callbacks=callbacks,
                    validation_steps=STEPS_VALID
                    )

Epoch 1/2
 31/260 [==>...........................] - ETA: 4:08 - loss: 0.0535 - iou_score: 0.9718 - f1-score: 0.9855   

In [18]:

print('[MODEL PRUNED]')
sub_model_for_prun.set_weights(model_for_prun.layers[-1].get_weights())

model_for_export = tfmot.sparsity.keras.strip_pruning(sub_model_for_prun)

print("final model")
model_for_export.summary()


[MODEL PRUNED]
final model
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, None, None, 4 1080        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, None, None, 4 160         stem_conv[1][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, None, None, 4 0           stem_bn[1][0]                    
___________________________________________________________________

In [19]:

import numpy as np

for i, w in enumerate(model_for_export.get_weights()):
    print(
        "{} -- Total:{}, Zeros: {:.2f}%".format(
            model_for_export.weights[i].name, w.size, np.sum(w == 0) / w.size * 100
        )
    )

stem_conv_1/kernel:0 -- Total:1080, Zeros: 79.63%
stem_bn_1/gamma:0 -- Total:40, Zeros: 0.00%
stem_bn_1/beta:0 -- Total:40, Zeros: 0.00%
stem_bn_1/moving_mean:0 -- Total:40, Zeros: 0.00%
stem_bn_1/moving_variance:0 -- Total:40, Zeros: 0.00%
block1a_dwconv_1/depthwise_kernel:0 -- Total:360, Zeros: 0.00%
block1a_bn_1/gamma:0 -- Total:40, Zeros: 0.00%
block1a_bn_1/beta:0 -- Total:40, Zeros: 0.00%
block1a_bn_1/moving_mean:0 -- Total:40, Zeros: 0.00%
block1a_bn_1/moving_variance:0 -- Total:40, Zeros: 0.00%
block1a_se_reduce_1/kernel:0 -- Total:400, Zeros: 79.75%
block1a_se_reduce_1/bias:0 -- Total:10, Zeros: 0.00%
block1a_se_expand_1/kernel:0 -- Total:400, Zeros: 79.75%
block1a_se_expand_1/bias:0 -- Total:40, Zeros: 0.00%
block1a_project_conv_1/kernel:0 -- Total:960, Zeros: 79.58%
block1a_project_bn_1/gamma:0 -- Total:24, Zeros: 0.00%
block1a_project_bn_1/beta:0 -- Total:24, Zeros: 0.00%
block1a_project_bn_1/moving_mean:0 -- Total:24, Zeros: 0.00%
block1a_project_bn_1/moving_variance:0 -- T

In [20]:
final_model = compile_model(model_for_export)

In [ ]:
import tempfile

def get_gzipped_model_size(model):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, keras_file = tempfile.mkstemp('.h5')
  model.save(keras_file, include_optimizer=False)

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(keras_file)

  return os.path.getsize(zipped_file)

print("Size of gzipped pruned model without stripping: %.2f bytes" % (get_gzipped_model_size(model)))
print("Size of gzipped pruned model with stripping: %.2f bytes" % (get_gzipped_model_size(final_model)))

In [21]:
final_model.save('pruned_model_2.h5')

In [ ]:
test_model = keras.models.load_model('pruned_model.h5', compile=False)

In [ ]:
optim = keras.optimizers.Adam(LR)

# Segmentation models losses can be combined together by '+' and scaled by integer or float factor
dice_loss = sm.losses.DiceLoss()
focal_loss = sm.losses.BinaryFocalLoss() if N_CLASSES == 1 else sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]
test_model.compile(optim, total_loss, metrics)

In [ ]:
print('[TRAINING THE MODEL]')

test_model.fit_generator(train_dataloader,
                    steps_per_epoch=STEPS_PER_EPOCH,
                    epochs=FIRST_TRAIN_EPOCHS,
                    validation_data=valid_dataloader,
                    callbacks=callbacks,
                    validation_steps=STEPS_VALID
                    )

print('[MODEL PRETRAINED]')

In [ ]:
test_model.summary()

In [ ]:
test_dataset = Dataset(
    test_images,
    test_annotations,
    classes=CLASSES,
    augmentation=get_validation_augmentation(),
    #preprocessing=get_preprocessing(preprocess_input),
)

test_dataloader = Dataloder(test_dataset, batch_size=1, shuffle=False)

ids = np.arange(len(test_dataset))


for i in ids:
    image, gt_mask = test_dataset[i]
    image = np.expand_dims(image, axis=0)
    image = np.expand_dims(image, axis=3)
    pr_mask = test_model.predict(image).round()
# img_kek = train_dataloader[0][0]
# pr_mask = model.predict(img_kek).round()
# plt.figure(figsize = (28,28))
# plt.imshow(img_kek[..., 0].squeeze(),cmap='gray')
# plt.show()
# print(np.max(img_kek))
#     cv2.imshow('mask', image[0][:][:])
#     cv2.waitKey(0)

    visualize(
        image=denormalize(image.squeeze()),
        gt_mask=gt_mask[..., 0].squeeze(),
        pr_mask=pr_mask[..., 0].squeeze(),
    )
